## test find

In [ ]:
# test find(e,element_list)

import random
# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))
    def __repr__(self):
        return '[total count: {}, distinct: {}]'.format(self.count,len(self.distinct))

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
# ==========================data structure==========================

def find(e,element_list):
    try:
        index = [ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index

filename='kosarak_ground_truth.csv'
filepath="..\\result\\kosarak\\"
src_data=os.path.join(filepath,filename)

df=pd.read_csv(src_data)
x=[Tail(i,j) for i,j in df.values.tolist()]
y=[]
randomindex=[random.randint(1,len(x)) for i in range(100)]
for index in randomindex:
    y.append(x[index])
for item in y:
    result=find(item,x)
    print("item in y: {},find index:{},item in x[{}]:{}".format(item,result,result,x[result]))
    print("item?=x[result]:{}".format(item==x[result]))

## UpdateSK, hash-append-ID ver.

In [1]:
# 20210407
import spookyhash
import mmh3
from numpy import random
import os
import time
import operator
import hyperloglog

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}, max: {}]'.format(self.count,len(self.distinct),self.maxID)
    def __repr__(self):
        return '[total count: {}, distinct: {}, max: {}]'.format(self.count,len(self.distinct),self.maxID)

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)

# ==========================UpdateSk==========================
def UpdateSk(element,Sk_head,Sk):
    # print("In UpdateSk()")
    e_max=get_emax()
    ID,row=position(element)
        # row position of e and ID=h(e) in Sketch 
    avg=0
    item=Tail(ID, element.count)
        # ID in Sketch is hash value
    #print("{} -> {},send to Sk[{}]".format(element,item,row))
    # ==========================update sketch==========================
    index=find(item,Sk[row])
        # find index of e in Sk[row]
    Sk_head[row].count+=item.count
    if index >=0:
        # e matches in Sk[row]
        Sk[row][index].count+=item.count
    else:
        # e doesn't match in Sk[row]
        if len(Sk[row])<width:
            Sk[row].append(item)
            index=len(Sk[row])-1
        else:
            Sk_head[row].distinct.add(element.ID)
    Update_local_max(Sk_head[row],Sk[row],element,index)
    Update_emax(Sk_head,Sk)
    
    # now we have:
    # ID: h2(h1(e)), hash value of e
    # row: h1(e), row index of e
    # index: column index of e
    # Sk_head[row].count: total count of Sk[row]
    # Sk_head[row].maxID: local max count element of Sk[row]
    # Sk_head[row].distinct: estimated element of Other part in Sk[row]
    # count_sum: sum of count in Sketch[row]
    # avg: average count of Other part in Sk[row]

    
    # Sk[row].sort(key=operator.attrgetter('count'),reverse=True)
    # print("e_max:{}".format(e_max))
    # print("Sk_head[{}]:{}".format(row,Sk_head[row]))
    
    '''
    print("e_max:{}".format(e_max))
    for i in range(len(Sk)):
        print("Sk[{}]:{},{}".format(i,Sk_head[i],Sk[i]))
    print('')    
    '''


# ==========================update local max==========================       
def Update_local_max(head_item,element_list,element,index):
    #print("In Update_local_max, Sk[row]:{}".format(element_list))
    numertor,denominator=get_fraction()
    width,depth=get_width_depth()
    # distinct=len(head_item.distinct)
    if head_item.maxID=='':
        head_item.maxID=element.ID
    else:
        local_max_ID=(mmh3.hash(head_item.maxID, signed=False))% ((width*numertor)//denominator)
        local_max_index=find(Tail(local_max_ID,1),element_list)
        # print("local_max_index:{}".format(local_max_index))
        if local_max_index>=0:
            if index>=0:
                if element_list[index].count >element_list[local_max_index].count:
                    head_item.maxID=element.ID
            else:
                # index=-99
                count_sum=sum(element_list[i].count for i in range(len(element_list)))
                try:
                    avg=(head_item.count-count_sum)//len(head_item.distinct)
                except ZeroDivisionError:
                    print("len(head_item.distinct)={}".format(distinct))
                else:
                    if avg>element_list[local_max_index].count:
                        head_item.maxID=element.ID
        else:
            #　local_max_inde＝-99, local max is in Other
            count_sum=sum(element_list[i].count for i in range(len(element_list)))
            try:
                avg=(head_item.count-count_sum)//len(head_item.distinct)
            except ZeroDivisionError:
                print("len(head_item.distinct)={}".format(distinct))
            else:
                if index>=0:
                    if element_list[index].count >avg:
                        head_item.maxID=element.ID
                else:
                    head_item.maxID=element.ID
 # ==========================update e_max==========================
def Update_emax(head,sketch):
    e_max=get_emax()
    numerator,denominator=get_fraction()
    width,depth=get_width_depth()
    for i in range(len(head)):
        if head[i].maxID=='':
            continue
        else:
            local_max_ID=(mmh3.hash(head[i].maxID, signed=False))% ((width*numerator)//denominator)
            local_max_index=find(Tail(local_max_ID,1),sketch[i])
            if local_max_index>=0:
                if sketch[i][local_max_index].count>e_max.count:
                    e_max.ID=head[i].maxID
                    e_max.count=sketch[i][local_max_index].count

    
# ==========================Tools=========================    
def get_emax():
    return e_max
def get_width_depth():
    return width,depth

def find(e,element_list):
    # return index of e in element_list
    try:
        index=[ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index

def position(element):
    numertor,denominator=get_fraction()
    width,depth=get_width_depth()
    hash1=spookyhash.hash32(bytes(str(element.ID),encoding='utf-8'))
        # input of spooky: byte
        # output of spooky:unsigned- 32 bit int
    hash2=mmh3.hash(element.ID, signed=False)
        # input of mmh: str
        # output: unsigned- 32 bit int
    ID=hash2 % ((width*numertor)//denominator)
    row=hash1 % depth
    return ID,row
def get_fraction():
    return numerator,denominator

# ==========================main=========================

filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)
depth=4
width=128
size=1024
numerator=12
denominator=10

start=time.time()
Sk_head=[Head(0) for j in range(depth)]
Sk=[[] for i in range(depth)]

e_max=Tail('',0)

with open(src_data,'r') as file:
    while True:
        element=file.readline().strip('\n')
        if not element:
            break
        else:
            #item_count-=1
            item=Tail(element,1)
            UpdateSk(item,Sk_head,Sk)
end=time.time()
print("Execution time:{} seconds.".format(str(end-start)))
print("e_max:{}".format(e_max))
for i in range(len(Sk)):
    print("Sk[{}]:{},{}".format(i,Sk_head[i],Sk[i]))
print('') 

Execution time:826.2504391670227 seconds.
e_max:[ID: 3, count: 37374]
Sk[0]:[total count: 223820, distinct: 24846, max: 11 6],[[ID: 51, count: 1446], [ID: 111, count: 1365], [ID: 20, count: 1333], [ID: 24, count: 1119], [ID: 0, count: 1621], [ID: 136, count: 1255], [ID: 69, count: 1367], [ID: 43, count: 1333], [ID: 130, count: 1229], [ID: 122, count: 1320], [ID: 99, count: 1314], [ID: 71, count: 1374], [ID: 93, count: 1264], [ID: 61, count: 1180], [ID: 126, count: 1201], [ID: 12, count: 1337], [ID: 66, count: 1466], [ID: 25, count: 1116], [ID: 55, count: 1255], [ID: 87, count: 1272], [ID: 32, count: 1285], [ID: 19, count: 1166], [ID: 60, count: 1242], [ID: 29, count: 14231], [ID: 48, count: 1200], [ID: 2, count: 1173], [ID: 140, count: 1690], [ID: 119, count: 1303], [ID: 31, count: 1308], [ID: 76, count: 1157], [ID: 131, count: 1170], [ID: 152, count: 2079], [ID: 84, count: 1129], [ID: 9, count: 1345], [ID: 33, count: 1184], [ID: 42, count: 2315], [ID: 73, count: 1314], [ID: 6, count: 

## UpdateSK, hash-indexing version

In [1]:
# hash-map array version.
import numpy as np
import spookyhash
import mmh3
import os
import pandas as pd
import time
import operator
import hyperloglog
import sys

# ==========================data structure==========================
class Node():
    def __init__(self,count=0):
        self.count=count
    def add_count(self,count=1):
        self.count+=count
    def __str__(self):
        return 'count: {}'.format(self.count)
    def __repr__(self):
        return ''

class Head(Node):
    def __init__(self,count=1):
        super().__init__(count)
        self.distinct = hyperloglog.HyperLogLog(0.01)
        self.maxID=''
    def __str__(self):
        return '[total count: {}, distinct: {}, max: {}]'.format(self.count,len(self.distinct),self.maxID)
    def __repr__(self):
        return '[total count: {}, distinct: {}, max: {}]'.format(self.count,len(self.distinct),self.maxID)

class Tail(Node):
    def __init__(self,ID,count):
        self.ID = ID
        super().__init__(count)
    def __str__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)
    def __repr__(self):
        return '[ID: {}, count: {}]'.format(self.ID,self.count)


# ==========================UpdateSk==========================
def UpdateSk(element,Sk_head,Sk):
    e_max=get_emax()
    width,depth=get_width_depth()
    numerator,denominator=get_fraction()
    col,row=position(element)
        # col, row: index of element in sketch
    avg=0
        # ID in Sketch is hash value
    # ==========================update sketch==========================
    Sk_head[row].count+=element.count
    if col<width:
        # e in Sketch
        Sk[row][col]+=1
    else:
        # e in Other
        count_sum=sum(i for i in Sk[row])
        avg=(Sk_head[row].count-count_sum)//(width*((numerator/denominator)-1))
    Update_local_max(Sk_head[row],Sk[row],element,col)
    Update_emax(Sk_head,Sk)

    '''
    print("e_max:{}".format(e_max))
    for i in range(len(Sk)):
        print("Sk[{}]:{},{}".format(i,Sk_head[i],Sk[i]))
    print('')
    '''

# ==========================update local max==========================       
def Update_local_max(head_item,element_list,element,column):
    # pass single row
    numerator,denominator=get_fraction()
    width,depth=get_width_depth()
    if head_item.maxID=='':
        head_item.maxID=element.ID
    else:
        local_max_col=(mmh3.hash(head_item.maxID,signed=False))% ((width*numerator)//denominator)
        if local_max_col<width:
            # local max in Sketch
            if column<width:
                # e in Sketch
                if element_list[column]>element_list[local_max_col]:
                       head_item.maxID=element.ID
            else:
                # e in Other
                count_sum=sum(i for i in element_list)
                avg=(head_item.count-count_sum)//(width*((numerator/denominator)-1))
                if avg>element_list[local_max_col]:
                     head_item.maxID=element.ID
        else:
            # local max in Other
            count_sum=sum(i for i in element_list)
            avg=(head_item.count-count_sum)//(width*((numerator/denominator)-1))  
            if column<width:
                if column<width:
                    if element_list[column]>avg:
                           head_item.maxID=element.ID
                else:
                    pass

# ==========================update e_max==========================
def Update_emax(head,sketch):
    # pass whole array
    e_max=get_emax()
    numerator,denominator=get_fraction()
    width,depth=get_width_depth()
    for i in range(len(head)):
        if head[i].maxID=='':
            continue
        else:
            local_max_col,local_max_row=position(Tail(head[i].maxID,0))
            if local_max_col<width:
                if sketch[local_max_row][local_max_col]>e_max.count:
                    e_max.ID=head[i].maxID
                    e_max.count=sketch[local_max_row][local_max_col]
            else:
                # local max in Other
                count_sum=sum(j for j in sketch[i])
                avg=(head[i].count-count_sum)//(width*((numerator/denominator)-1))
                if avg>e_max.count:
                    e_max.ID=head[i].maxID
                    e_max.count=avg
# ==========================Tools=========================    
def get_emax():
    return e_max
def get_width_depth():
    return width,depth

def find(e,element_list):
    # return index of e in element_list
    try:
        index=[ele.ID for ele in element_list].index(e.ID)
    except:
        index=-99
    return index

def position(element):
    numerator,denominator=get_fraction()
    width,depth=get_width_depth()
    hash1=spookyhash.hash32(bytes(str(element.ID),encoding='utf-8'))
        # input of spooky: byte
        # output of spooky:unsigned- 32 bit int
    hash2=mmh3.hash(element.ID, signed=False)
        # input of mmh: str
        # output: unsigned- 32 bit int
    col=hash2 % ((width*numerator)//denominator)
    row=hash1 % depth
    return col,row
def get_fraction():
    return numerator,denominator    
    
# ==========================main=========================    

filename='kosarak.dat'
filepath="..\\dataset\\"
src_data=os.path.join(filepath,filename)
depth=4
width=128
size=1024
numerator=12
denominator=10

start=time.time()
Sk_head=[Head(0) for j in range(depth)]
Sketch=np.zeros((depth,width),dtype='int16')
e_max=Tail('',0)

item_count=100
income=0
with open(src_data,'r') as file:
    while True:
        e=file.readline().strip('\n')
        if not e:
            break
        else:
            #item_count-=1
            # income+=1
            # print("read {}-th element:".format(income))
            item=Tail(e,1)
            UpdateSk(item,Sk_head,Sketch)
end=time.time()
print("Execution time:{} seconds.".format(str(end-start)))
print("e_max:{}".format(e_max))
for i in range(len(Sketch)):
    print("Sk[{}]:{},{}".format(i,Sk_head[i],Sketch[i]))
print('') 

Execution time:46.107290744781494 seconds.
e_max:[ID: 3, count: 37374]
Sk[0]:[total count: 223820, distinct: 0, max: 11 6],[ 1621  1819  1173  1244  1380  1316  1557  4447  1216  1345  1530  1927
  1337  1374  1152  1234  1226  1086  1126  1166  1333  1110  1318  1146
  1119  1116  1148  1185  1298 14231  1471  1308  1285  1184  1122  2377
  1702  1216  1338  1313  1420  1240  2315  1333  1348  1170  1114  1251
  1200  1198  1219  1446  1401  1162  1496  1255  1276  1386  1172  1166
  1242  1180  1352  1194  1419  1240  1466  2899  1229  1367  2240  1374
  1273  1314  1205  1339  1157  1154  1788  1537  1157  1127  1512  1318
  1129  1661  1177  1272  4984  1395  1259  1202  1164  1264  1215  1371
  1601  1159  1279  1314  1203  1344  1787  1166  1255  1209  1283  1176
  1209  1290  1560  1365  1221  1318  1474  1185  1326  1369  1466  1303
  1194  1287  1320  1309  1152  1261  1201  1086]
Sk[1]:[total count: 228615, distinct: 0, max: 1],[ 1392  1595  1304  1368  1237  1369  1334  1572

## DeleteSK

In [ ]:


def DeleteSk(element):
    # Delete element in SK and update e_max
    hash1=spookyhash.hash32(bytes((element.ID),encoding='utf-8'))
    hash2=mmh3.hash(str(hash1), signed=False)
    ID=hash2 % ((width*3)//2)
    row=hash1 % depth
    Sk[row][0].count-=element.count
    index=find(ID,Sk[row])
    if index<0:
        pass
    else:
        Sk[row].pop(index)


filename='kosarak_ground_truth.csv'
filepath="..\\result\\kosarak\\"
src_data=os.path.join(filepath,filename)
df=pd.read_csv(src_data)
x=[Tail(i,j) for i,j in df.values.tolist()]

depth=4
width=128
Sk=[[ds.Head(0)] for j in range(depth)]


In [ ]:

import copy

x=copy.copy(Sk[3])
print(id(x),id(Sk[3]))
print(x)

def sort_Sk(element_list):
    tail=element_list[1:]
    tail.sort(key=operator.attrgetter('count'),reverse=True)
    return [element_list[0]]+tail
    
Sk[3]=sort_Sk(x)
print(Sk[3])